In [1]:
import sys 
## para inserir o modulo src
sys.path.insert(0, '../src/')

In [2]:
## importa a função de carregamento dos dados
from data.data_load import DataLoad
### nome pasta.função import classe

In [3]:
### instaciando a classe de leitura dos dados
dl = DataLoad()
df = dl.data_load('train_dataset_name')

2024-03-24 20:04:39 [info     ] Comecando a carga dos dados com o nome: train_dataset_name


In [4]:
df.head()

,target,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [5]:
df.shape

(150000, 11)

In [6]:
from data.data_validation import DataValidation

In [7]:
dv = DataValidation()

is_valid = dv.run(df)

2024-03-24 20:04:44 [info     ] Validacao iniciou             
2024-03-24 20:04:44 [info     ] Validation columns passed...  
2024-03-24 20:04:45 [info     ] Validacao com sucesso.        


In [8]:
from data.data_transformation import DataTransformation

In [9]:
## instanciando a classe
dt = DataTransformation(df)

X_train, X_valid, y_train, y_valid = dt.train_test_spliting()

In [10]:
X_train.head()

,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
138985,0.000000,49,0,0.127227,5163.0,5,0,0,0,0.0
63964,0.033105,37,0,0.423063,2800.0,4,0,1,0,1.0
46077,0.007948,39,0,0.685663,3333.0,11,0,1,0,0.0
83030,0.576297,57,0,0.320077,15161.0,14,0,4,0,0.0
11184,0.623724,35,0,0.534226,6500.0,9,0,3,0,1.0


In [11]:
from sklearn.pipeline import Pipeline
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler

from utils.utils import load_config_file

In [12]:
pipe = Pipeline([('imputer', MeanMedianImputer(variables=load_config_file().get('vars_imputer'))),
                 ('discretizer', EqualFrequencyDiscretiser(variables=load_config_file().get('vars_discretizer'))),
                 ('scaler', SklearnTransformerWrapper(StandardScaler()))])

In [13]:
from data.data_preprocess import DataPreprocess

In [14]:
preprocessor = DataPreprocess(pipe)

In [15]:
preprocessor.train(X_train)

2024-03-24 20:04:48 [info     ] Preprocessamento iniciou...   


In [16]:
X_train_processed = preprocessor.transform(X_train)

2024-03-24 20:04:49 [info     ] Transformacao dos dados com preprocessador iniciou...


In [17]:
X_valid_processed = preprocessor.transform(X_valid)

2024-03-24 20:04:49 [info     ] Transformacao dos dados com preprocessador iniciou...


In [18]:
X_train_processed.shape

(105000, 10)

In [19]:
X_valid_processed.shape

(45000, 10)

In [20]:
from train.train import TrainModels

In [21]:
tm = TrainModels(dados_X=X_train_processed,
                 dados_y=y_train)

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
model = tm.train(model=LogisticRegression())

In [24]:
from evaluation.classifier_eval import ModelEvaluation

In [25]:
model_eval = ModelEvaluation(model,
                             X_train_processed,
                             y_train,
                             n_splits=5)
roc_auc_scores = model_eval.cross_val_evaluate()

2024-03-24 20:04:52 [info     ] Iniciou a validacao cruzada...


In [28]:
roc_auc_scores.mean()

0.8059241681147551